In [1]:
import logging
import json
import torch
from types import SimpleNamespace
from vilbert.vilbert import VILBertActionSelection, BertConfig
from pytorch_transformers.tokenization_bert import BertTokenizer
from pytorch_transformers.optimization import AdamW, WarmupLinearSchedule
import torch.distributed as dist
from VLN_config import config as args
from torch.nn import CrossEntropyLoss
import random
import pandas as pd
from dataLoaderActSelection import DataLoader
from torch.utils.tensorboard import SummaryWriter


In [ ]:
import sys
import os
import torch
import yaml

import numpy as np
import matplotlib.pyplot as plt
import PIL

from PIL import Image
import cv2
import argparse
import glob
import pdb

import torchvision.models as models
import torchvision.transforms as transforms

from faster_rcnn import feature_extractor_new as f_extractor
from faster_rcnn.feature_extractor_new import featureExtractor
#%matplotlib inline  

In [2]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

In [4]:
config = BertConfig.from_json_file(args.config_file)
bert_weight_name = json.load(
    open("config/" + args.bert_model + "_weight_name.json", "r")
)

tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

config.track_temporal_features = args.track_temporal_features
config.mean_layer = args.mean_layer
config.max_temporal_memory_buffer = args.max_temporal_memory_buffer
config.visualization = True

model = VILBertActionSelection.from_pretrained(
    args.from_pretrained, config=config, default_gpu=True
)

07/15/2020 13:50:57 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/salem/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
07/15/2020 13:50:57 - INFO - vilbert.utils -   loading weights file save/multitask_model/multi_task_model.bin
07/15/2020 13:51:02 - INFO - vilbert.utils -   Weights of VILBertActionSelection not initialized from pretrained model: ['positional_enc.weight', 'positional_enc.bias', 'img_emb_mean.weight', 'img_emb_mean.bias', 'vil_prediction_action_selection.logit_fc.0.weight', 'vil_prediction_action_selection.logit_fc.0.bias', 'vil_prediction_action_selection.logit_fc.2.weight', 'vil_prediction_action_selection.logit_fc.2.bias', 'vil_prediction_action_selection.logit_fc.3.weight', 'vil_prediction_action_selection.logit_fc.3.bias']
07/15/202

In [5]:
model

VILBertActionSelection(
  (positional_enc): Linear(in_features=7, out_features=2048, bias=True)
  (img_emb_mean): Linear(in_features=10240, out_features=2048, bias=True)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (v_embeddings): BertImageEmbeddings(
      (image_embeddings): Linear(in_features=2048, out_features=1024, bias=True)
      (image_location_embeddings): Linear(in_features=5, out_features=1024, bias=True)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True

In [5]:
for key, value in dict(model.named_parameters()).items():
    if not value.requires_grad:
        print("This parameter does have grad", key)

In [ ]:
# load data
frcnn_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
data_loader = DataLoader("json_data.json", frcnn_model, save_or_not = False)
path = 'DataLoaderActSelection.pt'
data_loaded = data_loader.load_dataloader(path)


In [ ]:
def split_train_val (data_loaded, split_portion = 0.9):
    # WE are going to ignore infos
    features, pos_enc, spatial, image_mask, tokenized_text, input_mask, segment_ids, co_attention_mask, infos, action_targets  = data_loaded
    
    indexes = list(range(features.shape[0]))
    random.shuffle(indexes)
    split_train = int(features.shape[0] * split_portion)
    indexes_train = indexes[:split_train]
    indexes_val = indexes[split_train:]
    data_loaded_train = (features[indexes_train], 
                         pos_enc[indexes_train], 
                         spatial[indexes_train], 
                         image_mask[indexes_train], 
                         tokenized_text[indexes_train], 
                         input_mask[indexes_train], 
                         segment_ids[indexes_train], 
                         co_attention_mask[indexes_train],  
                         action_targets[indexes_train])
    data_loaded_val = (features[indexes_val], 
                         pos_enc[indexes_val], 
                         spatial[indexes_val], 
                         image_mask[indexes_val], 
                         tokenized_text[indexes_val], 
                         input_mask[indexes_val], 
                         segment_ids[indexes_val], 
                         co_attention_mask[indexes_val],  
                         action_targets[indexes_val])
    return data_loaded_train, data_loaded_val 

In [ ]:
model.cpu()
model.train()
optimizer = AdamW(model.parameters(),
                    lr=args.learning_rate,
                    eps=args.adam_epsilon,
                    betas=(0.9, 0.98),)

In [ ]:
criterion = CrossEntropyLoss()
batch_size = args.train_batch_size 
if args.use_tensorboard:
    writer = SummaryWriter()
data_train , data_val = split_train_val(data_loaded)
(features_train, pos_enc_train, spatial_train, image_mask_train, tokenized_text_train, input_mask_train, 
    segment_ids_train, co_attention_mask_train, action_targets_train)  = data_train

(features_val, pos_enc_val, spatial_val, image_mask_val, tokenized_text_val, input_mask_val, 
    segment_ids_val, co_attention_mask_val, action_targets_val)  = data_val
loss_result_csv = pd.DataFrame(columns = ['epochs', 'train_loss', 'val_loss'])

In [ ]:
for epoch in range(2):
    i = 0
    loss_train_cum = 0.
    num_batches = data_train[0].shape[0]//batch_size+1
    #import pdb; pdb.set_trace()
    for i in range(num_batches):
        if (i == num_batches -1):
            r =  data_train[0].shape[0]%batch_size
            pred_action_train, att_train = model(input_ids = tokenized_text_train[data_train[0].shape[0]-batch_size:].cpu(),
                                image_feat = features_train[data_train[0].shape[0]-batch_size:].cpu(), # Linear(2048*config.max_temporal_memory_buffer, 2048)
                                image_loc = spatial_train[data_train[0].shape[0]-batch_size:].cpu(),  #Linear(in_features=5, out_features=1024, bias=True)
                                image_pos_input = pos_enc_train[data_train[0].shape[0]-batch_size:].cpu(),   #Linear(7, 2048)/(6, 2048)
                                token_type_ids = segment_ids_train[data_train[0].shape[0]-batch_size:].cpu(), 
                                attention_mask = input_mask_train[data_train[0].shape[0]-batch_size:].cpu(), 
                                image_attention_mask = image_mask_train[data_train[0].shape[0]-batch_size:].cpu(),
                                output_all_attention_masks=True)
            # Check the shapes in the criterion
            loss_train = criterion(pred_action_train, action_targets_train[data_train[0].shape[0]-batch_size:].cpu())
        else:
            pred_action_train, att_train = model(input_ids = tokenized_text_train[i*batch_size:(i+1)*batch_size].cpu(),
                                image_feat = features_train[i*batch_size:(i+1)*batch_size].cpu(), # Linear(2048*config.max_temporal_memory_buffer, 2048)
                                image_loc = spatial_train[i*batch_size:(i+1)*batch_size].cpu(),  #Linear(in_features=5, out_features=1024, bias=True)
                                image_pos_input = pos_enc_train[i*batch_size:(i+1)*batch_size].cpu(),   #Linear(7, 2048)/(6, 2048)
                                token_type_ids = segment_ids_train[i*batch_size:(i+1)*batch_size].cpu(), 
                                attention_mask = input_mask_train[i*batch_size:(i+1)*batch_size].cpu(), 
                                image_attention_mask = image_mask_train[i*batch_size:(i+1)*batch_size].cpu(),
                                output_all_attention_masks=True)
            # Check the shapes in the criterion
            import pdb; pdb.set_trace()
            loss_train = criterion(pred_action, action_targets_train[i*batch_size:(i+1)*batch_size].cpu().view(-1))        
        optimizer.zero_grad()
        loss_train.backward()
        loss_train_cum += loss_train
        optimizer.step()
    loss_train_cum = loss_train_cum/data_train[0].shape[0]
    print("epoch: " , epoch, " Train loss: ", loss_train_cum)
    # Validation
    pred_action_val, att_val = model(input_ids = tokenized_text_val.cpu(),
                            image_feat = features_val.cpu(), # Linear(2048*config.max_temporal_memory_buffer, 2048)
                            image_loc = spatial_val.cpu(),  #Linear(in_features=5, out_features=1024, bias=True)
                            image_pos_input = pos_enc_val.cpu(),   #Linear(7, 2048)/(6, 2048)
                            token_type_ids = segment_ids_val.cpu(), 
                            attention_mask = input_mask_val.cpu(), 
                            image_attention_mask = image_mask_val.cpu(),
                            output_all_attention_masks=True)
    optimizer.zero_grad()
    # Check the shapes in the criterion
    loss_val = criterion(pred_action_val, action_targets_val.cpu())
    loss_val = loss_val / data_val[0].shape[0]
    print("epoch: " , epoch,"Train loss: ", loss_train_cum.item(), " Val loss: ", loss_val.item())
    if args.use_tensorboard:
        #Plot separately the losses img and lm
        writer.add_scalar('Loss/train', loss_train_cum, epoch)
        writer.add_scalar('Loss/validation', loss_val, epoch)
    loss_result_csv = loss_result_csv.append(pd.DataFrame([[epoch, loss_train_cum.item(), loss_val.item()]], columns = loss_result_csv.columns ), ignore_index = True)    
    if epoch % 10 == 0:
        #loss_result_csv.to_csv('epoch_loss_2.csv')
        #model.save_pretrained('save_vilbert_action_grounding')
        torch.save(model.state_dict(), "save_vilbert_action_selection/vilberActionSelection.bin")
        print("Model saved!")

writer.close()